# GLMのモデル選択
よいモデルとは？  
当てはまりのよいモデル?よい予測をするモデル?  
- 変数を増やせば増やすほど当てはまりのよいモデルになるが、過学習を引き起こす
- テストデータへの当てはまりだけを追求したモデルは、汎用的でなく、使えない
- よいモデルとはよい予測をするモデルである。

AICとは
- よい予測をするモデルがよいモデルであるとの考えに基づいて設計された選択基準

※ここで言うあてはまりのよさは、最大尤度が大きいということ

# 統計モデルの当てはまりの悪さ：逸脱度

## 逸脱度(deviance)とは
- 最大対数尤度を変形した統計量

最大対数尤度を$ logL^{*} $とすると以下で与えられる
$$
逸脱度 D = -2logL^{*}
$$

- あてはまりの悪さを表現する指標であり、数値が大きいならあてはまりが悪い

| 名前 | 定義 |
| :--- | :--- |
| 逸脱度(D) | $ -2logL^{*} $ |
| 最小の逸脱度 | フルモデルを当てはめたときのD |
| 残差逸脱度 | D - 最小のD |
| 最大の逸脱度 | NullモデルをあてはめたときのD |
| Null逸脱度 | 最大のD - 最小のD |

**フルモデルを当てはめたときのDとは**  
仮に被説明変数$y_i$が100個のデータとしたら、
- i\in{1,2,3}の$ y_i $は6なので、{$ \lambda_1,\lambda_2,\lambda_3 $}={6,6,6}
- i=4の$ y_4 $は12なので$ \lambda_4=12 $
- 以下略

上記の様に100個のパラメータに対して、100個の被説明変数を読み上げる様に、値を決定した場合の逸脱度が最小の逸脱度となる  
**最大の逸脱度とは**  
最もあてはまりの悪いモデルで、ポアソン回帰の場合、$ y_i=e^{\beta_i} $のような切片だけのモデルのときでの逸脱度である.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as scs
%matplotlib inline

In [2]:
data = pd.read_csv('data3a.csv')
data

,y,x,f
0,6,8.31,C
1,6,9.44,C
2,6,9.50,C
3,12,9.07,C
4,10,10.16,C
...,...,...,...
95,8,9.15,T
96,6,8.52,T
97,8,10.24,T
98,7,10.86,T


In [3]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [4]:
model = smf.glm('y ~ x',data=data,family=sm.families.Poisson())
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                  100
Model:                            GLM   Df Residuals:                       98
Model Family:                 Poisson   Df Model:                            1
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -235.39
Date:                Mon, 26 Jul 2021   Deviance:                       84.993
Time:                        05:54:53   Pearson chi2:                     83.8
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.2917      0.364      3.552      0.000       0.579       2.005
x              0.0757      0.036      2.125      0.034       0.006       0.145
==============================================================================
"""

上記の結果について、残差逸脱度（D - 最小のD）が、Deviance:84.993で表示されている

## full modelの逸脱度（最小の逸脱度）の計算

In [5]:
# 無名関数の定義:変数 = lambda 引数:式
dpois = lambda x:scs.poisson.logpmf(x,x)

In [6]:
# 与えられたデータを最尤推定値としたポアソン分布の最大対数尤度を返す無名関数に、yの値を入れていく
dpois(data.y)

array([-1.8286944 , -1.8286944 , -1.8286944 , -2.1683347 , -2.07856164,
       -1.63287639, -2.02680628, -2.02680628, -2.02680628, -2.12545985,
       -1.8286944 , -2.07856164, -1.8286944 , -2.07856164, -2.12545985,
       -1.96907057, -1.4959226 , -1.96907057, -1.74030218, -1.74030218,
       -1.63287639, -2.12545985, -1.74030218, -2.07856164, -1.8286944 ,
       -1.8286944 , -1.90379032, -2.02680628, -1.4959226 , -2.07856164,
       -1.30685282, -2.02680628, -2.07856164, -1.74030218, -2.12545985,
       -2.07856164, -1.63287639, -1.96907057, -2.02680628, -2.1683347 ,
       -1.96907057, -2.02680628, -1.96907057, -1.8286944 , -1.8286944 ,
       -2.07856164, -2.07856164, -2.02680628, -2.1683347 , -1.8286944 ,
       -2.24441857, -1.8286944 , -1.90379032, -2.02680628, -1.8286944 ,
       -1.90379032, -2.02680628, -2.20782221, -2.02680628, -2.20782221,
       -1.90379032, -1.96907057, -2.07856164, -1.90379032, -2.1683347 ,
       -1.8286944 , -2.27851837, -1.4959226 , -1.63287639, -1.82

In [7]:
# それぞれのiに対する最大対数尤度をすべて加算し、全体の最大対数尤度を算出
max_logl = sum(dpois(data.y))
max_logl

-192.8897525244958

In [8]:
# 最小逸脱度を計算
min_dev = max_logl * -2
min_dev

385.7795050489916

## null modelの作成
切片だけの最も最大対数尤度が低いモデル

In [9]:
model = smf.glm('y ~ 1',data=data,family=sm.families.Poisson())
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                  100
Model:                            GLM   Df Residuals:                       99
Model Family:                 Poisson   Df Model:                            0
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -237.64
Date:                Mon, 26 Jul 2021   Deviance:                       89.507
Time:                        05:54:53   Pearson chi2:                     87.1
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.0580      0.036     57.586      0.000       1.988       2.128
==============================================================================
"""

- 残差逸脱度はDeviance:89.507である
- 一定モデルの最大対数尤度は-237.64であり、逸脱度は-2をかけて475.28程度。最小逸脱度は385.77程度だったので、475.28 - 385.77 = 89.51程度でDevianceと一致している

## (確認)一定モデルなら\lambdaはデータ平均と一致する
$$log\lambda = \beta_1$$

In [10]:
data.y.mean()

7.83

In [11]:
np.log(data.y.mean())

2.057962510002712

データ平均のlogと、切片の推定値が一致している

# モデル選択基準AIC

## AIC(Akaike's information criterionとは
- 統計モデルの予測の良さを重視するモデル選択基準

最尤推定したパラメータ個数がKであるとき
$$
\begin{aligned}
  AIC &= -2logL^{*} + 2k \\
  &= D + 2k \\
\end{aligned}
$$
AICが一番小さいモデルがよいモデルとされる。


## ネストしているモデルとは?
- 一定モデル:$ log\lambda_i=\beta_1 $と、xモデル:$ log\lambda_i=\beta_1+\beta_2x_i $の様に一方が他方の含まれているモデルのこと。→xモデルにおいて、$ \beta_2=0 $とすると、一定モデルになる。

## なぜAICでモデル選択してよいのか
**平均対数尤度とは**  
真の分布が分かっていると仮定したとき、真の分布から何回もサンプリングしておく。はじめに推定されたパラメータを織り込んだモデルと、真の分布からサンプリングしたデータとの対数尤度を、何回も算定し、その平均をとったもの。よって、これが真の分布との真の対数尤度といえて、**統計モデルの予測の良さを示す量**となる。  
**バイアスとは**  
構築したモデルの最大対数尤度($ logL^{*} $)と、平均対数尤度($ E(logL) $)との差bをバイアスとして、以下で定義する。
$$
b = logL^{*} - E(logL)
$$
この時、知りたいのは平均対数尤度であるので、式変形すると
$$
E(logL) = logL^{*} - b
$$
上記の変形をバイアス補正という  
**なぜAICが予測の良さを示す量なのか**  
$$
E(logL) = logL^{*} - b
$$
において、b = kとなることが解析的、数学的に知られている。  
そのため、$ AIC = -2(logL^{*} - k) $は平均対数尤度に-2を掛けたものであり、数値が小さいほど、真の分布に対しての当てはまりがよい。つまり、真の分布に対しての予測が優れていると解釈することができる。

